In [2]:
import numpy as np
datos = np.load('ulises.npy')
datos

array([[   0,  509,  501,  312, 1019,  736,  656,   60, 1039,  726, 2314,
         479,  448,  479,  619,  150],
       [ 509,    0,  126,  474, 1526, 1226, 1133,  532, 1449, 1122, 2789,
         958,  941,  978, 1127,  542],
       [ 501,  126,    0,  541, 1516, 1184, 1084,  536, 1371, 1045, 2728,
         913,  904,  946, 1115,  499],
       [ 312,  474,  541,    0, 1157,  980,  919,  271, 1333, 1029, 2553,
         751,  704,  720,  783,  455],
       [1019, 1526, 1516, 1157,    0,  478,  583,  996,  858,  855, 1504,
         677,  651,  600,  401, 1033],
       [ 736, 1226, 1184,  980,  478,    0,  115,  740,  470,  379, 1581,
         271,  289,  261,  308,  687],
       [ 656, 1133, 1084,  919,  583,  115,    0,  667,  455,  288, 1661,
         177,  216,  207,  343,  592],
       [  60,  532,  536,  271,  996,  740,  667,    0, 1066,  759, 2320,
         493,  454,  479,  598,  206],
       [1039, 1449, 1371, 1333,  858,  470,  455, 1066,    0,  328, 1387,
         591,  650,  6

In [17]:
import random

# Número de ciudades
n = 16  # Ajusta el número de ciudades según tu problema
# Parmetros del algoritmo genético
Sc = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
tamano_poblacion = 20


indx = 0
num_generaciones = 0
xx = n-1
longitud_cromosoma= int(np.round(np.log2(np.math.factorial(xx))))

# Inicializar la población



# Función para generar una población inicial
def generar_poblacion(tamano_poblacion, longitud_cromosoma):
    return [''.join(random.choice('01') for _ in range(longitud_cromosoma)) for _ in range(tamano_poblacion)]

# Función para evaluar la aptitud de un individuo
def evaluar_aptitud(cromosoma):
    # Paso 1: Convertir el cromosoma en un número decimal v
    v = int(cromosoma, 2)

    # Paso 2: Asignar a c el número de ciudades menos 1
    c = n

    # Inicializar el vector S
    S = []

    # Evaluar la aptitud
    ias=0
    while c > 0:
        # Paso 3: Hallar el resto de la división entre v y c y añadírselo al vector S
        resto = v % c
        S.append(resto)

        # Paso 4: Asignarle a v el resultado entero de la división entre v y c
        v = v // c

        # Paso 5: Restarle 1 a c
        c -= 1

    # Paso 6: Si c es mayor que 1, volver al Paso 3
    Sn = []
    Scaux = Sc.copy()

    for posicion in S:

        valor_seleccionado = Scaux.pop(posicion)
        Sn.append(valor_seleccionado)
    for ciudad in Scaux:
        if not ciudad in Sn:
            Sn.append(ciudad)



    aptitud = calcular_distancia_total(Sn)

    return aptitud

# Función para calcular la distancia total recorrida (por ejemplo)
def calcular_distancia_total(individuo):
    # Aquí implementa el cálculo de la distancia total basado en el vector S
        ciudad_actual = individuo[0]  # Obtén la primera ciudad
        distancia_total = 0

        for i in range(1, 15):
            siguiente_ciudad = individuo[i]

            distancia_total += datos[ciudad_actual-1][siguiente_ciudad-1]
            ciudad_actual = siguiente_ciudad

        # Regresa a la ciudad de inicio
        distancia_total += datos[ciudad_actual-1][individuo[0]-1]

        return distancia_total




def seleccionar_individuo_ruleta(poblacion, aptitudes):
    total_aptitud = sum(aptitudes)
    probabilidad = [aptitud / total_aptitud for aptitud in aptitudes]
    return random.choices(poblacion, probabilidad)[0]

def seleccionar_individuo_torneo(poblacion, aptitudes):
    k=10
    selected_population = []
    population_size = len(poblacion)

    for _ in range(population_size):
        tournament = random.sample(range(population_size), k)  # Selecciona 'k' individuos al azar para el torneo
        winner = min(tournament, key=lambda i: aptitudes[i])  # Elige al individuo con la menor aptitud en el torneo
        selected_population.append(poblacion[winner])

    return selected_population

def tournament_selection(population, fitness_values):
    selected_parents = []
    k = 5
    num_parents = 2
    for _ in range(num_parents):
        tournament = random.sample(range(len(population)), k)
        winner = max(tournament, key=lambda i: fitness_values[i])
        selected_parents.append(population[winner])

    return selected_parents



def find_best_individual(population, fitness_values):
    # Encontrar y registrar al mejor individuo en la población.
    best_index = fitness_values.index(min(fitness_values))
    best_individual = population[best_index]

    return best_individual

def replace_worst_with_best(population, fitness_values, best_individual):
    # Encontrar el peor individuo y sustituirlo por el mejor individuo si es superior.
    worst_index = fitness_values.index(min(fitness_values))
    if evaluar_aptitud(best_individual) > fitness_values[worst_index]:
        population[worst_index] = best_individual

def cruzamiento_un_punto(padre1, padre2):
    punto_cruzamiento = random.randint(1, 40)  # Se elige un punto de cruzamiento aleatorio (excepto el primer y último bit)
   
    hijo1 = padre1[:punto_cruzamiento] + padre2[punto_cruzamiento:]
    hijo2 = padre2[:punto_cruzamiento] + padre1[punto_cruzamiento:]

    return hijo1, hijo2







# Función para aplicar la mutación a un individuo
def mutar_individuo(individuo, num_generaciones):
    probabilidad_mutacion = 0.1
    if num_generaciones > 500000:
        probabilidad_mutacion = probabilidad_mutacion/2
    if num_generaciones > 1000000:
        probabilidad_mutacion = probabilidad_mutacion/4
    if num_generaciones > 150000:
        probabilidad_mutacion = probabilidad_mutacion/8
    mutado = list(individuo)
    for i in range(len(individuo)):
        if random.random() < probabilidad_mutacion:
            # Cambia el bit si la probabilidad de mutación se cumple
            mutado[i] = '0' if individuo[i] == '1' else '1'
    return ''.join(mutado)

# Aplicar mutación a la población después del cruzamiento

def convmejor(cromosoma):
   # Paso 1: Convertir el cromosoma en un número decimal v
    v = int(cromosoma, 2)

    # Paso 2: Asignar a c el número de ciudades menos 1
    c = n

    # Inicializar el vector S
    S = []

    # Evaluar la aptitud
    ias=0
    while c > 0:
        # Paso 3: Hallar el resto de la división entre v y c y añadírselo al vector S
        resto = v % c
        S.append(resto)

        # Paso 4: Asignarle a v el resultado entero de la división entre v y c
        v = v // c

        # Paso 5: Restarle 1 a c
        c -= 1

    # Paso 6: Si c es mayor que 1, volver al Paso 3
    Sn = []
    Scaux = Sc.copy()

    for posicion in S:

        valor_seleccionado = Scaux.pop(posicion)
        Sn.append(valor_seleccionado)

    for ciudad in Scaux:
        if not ciudad in Sn:
            Sn.append(ciudad)


    return Sn
poblacion = generar_poblacion(tamano_poblacion, longitud_cromosoma)
# Algoritmo genético
while indx < 1:
    # Evaluación de aptitud

    # Selección
    nueva_poblacion = []


    for _ in range(tamano_poblacion // 2):
        padre1 = seleccionar_individuo_ruleta(poblacion, aptitudes)
        padre2 = seleccionar_individuo_ruleta(poblacion, aptitudes)

        hijo1, hijo2 = cruzamiento_un_punto(padre1, padre2)
        nueva_poblacion.extend([hijo1, hijo2])

    #for _ in range(tamano_poblacion // 2):
        #selected_parents = tournament_selection(poblacion, aptitudes)
        #hijo1, hijo2 = cruzamiento_un_punto(selected_parents[0], selected_parents[1])
        #nueva_poblacion.extend([hijo1, hijo2])
    # Mutación
    nueva_poblacion = [mutar_individuo(individuo,num_generaciones) for individuo in nueva_poblacion]
    
    # Reemplazar la población anterior con la nueva población
    tuplas = list(zip(poblacion, aptitudes))
    tuplas.sort(key=lambda x: x[1], reverse=False)
    poblacionant, aptitudesant = zip(*tuplas)
    
   
    poblacion = nueva_poblacion
    
    tuplas = list(zip(poblacion, aptitudes))
    tuplas.sort(key=lambda x: x[1], reverse=False)
    poblacion, aptitudes = zip(*tuplas)
    
    if aptitudes[15] > aptitudesant[0]:
        print(poblacion[15])
        print(poblacionant[15])
        poblacion[15] = poblacionant[0]
    if aptitudes[14] > aptitudesant[1]:
        poblacion[14] = poblacionant[1]
    

    #replace_worst_with_best(poblacion, aptitudes, mejor_individuo)

    # Mostrar la aptitud del mejor individuo en esta generación
    print(aptitudes)
    aptitudes = [evaluar_aptitud(cromosoma) for cromosoma in poblacion]
    tuplas = list(zip(poblacion, aptitudes))
    tuplas.sort(key=lambda x: x[1], reverse=False)
    poblacion, aptitudes = zip(*tuplas)
    print(aptitudesant)
    print(aptitudes)
    
    mejor_individuo_generacion = poblacion[0]
    mejor_aptitud_generacion = aptitudes[0]
    
    if(num_generaciones % 1 == 0): # Every 1 epochs


        print(f"Generación {num_generaciones}: Mejor individuo - {mejor_individuo_generacion}, Aptitud: {mejor_aptitud_generacion} mejor solucion: {convmejor(mejor_individuo_generacion)}")
    if mejor_aptitud_generacion < 4900 or num_generaciones == 200000: indx += 1
    num_generaciones += 1
print(f"mejor solucion: {convmejor(mejor_individuo_generacion)} Aptitud: {mejor_aptitud_generacion}")

1110001001011100110101010001110101001011
0111101011111111100101001100110010000001


TypeError: 'tuple' object does not support item assignment

In [ ]:
datos[12][10]


1868

In [ ]:
5 % 2

1